In [3]:
cd /tf/astromer

/tf/astromer


In [4]:
import tensorflow as tf
import pandas as pd
import json
import os

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from core.data  import from_generator
test_batches = from_generator(50, 100, 16)

In [15]:
for batch in test_batches:
    
    print(batch['mask_in'].shape)
    steps = tf.shape(batch['mask_in'])[1]
    mask_rshp = tf.tile(batch['mask_in'], [1,1,steps])
    mask_rshp += tf.transpose(mask_rshp, [0,2,1])
    mask_rshp = tf.minimum(1, mask_rshp)
    mask_rshp = tf.expand_dims(mask_rshp, 1)
    print(mask_rshp.shape)
    break

(16, 50, 1)
(16, 1, 50, 50)
